## Imports ##

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/2-preprocessed/cleanIPF.csv')

cols = ['Sex', 'Age', 'BodyweightKg', 'TotalKg']

# df = df[cols]
# df = df.dropna()

example_lifter = df[df['Name'] == 'Phillip Ngo'].copy()
print(example_lifter[['Date', 'Best3SquatKg', 'Best3BenchKg', 'Best3DeadliftKg', 'TotalKg', 'BodyweightKg']])


              Date  Best3SquatKg  Best3BenchKg  Best3DeadliftKg  TotalKg  \
157904  2024-01-27         107.5          70.0            145.0    322.5   
157905  2025-02-01         132.5          87.5            160.0    380.0   

        BodyweightKg  
157904          65.0  
157905          64.6  


## Feature engineering ##

In [ ]:
def create_features(current_meet, previous_meet):
    features = current_meet.copy()

    # 1. previous performance
    features['prev_squat'] = previous_meet['Best3SquatKg'].iloc[-1]
    features['prev_bench'] = previous_meet['Best3BenchKg'].iloc[-1]
    features['prev_deadlift'] = previous_meet['Best3DeadliftKg'].iloc[-1]
    features['prev_total'] = previous_meet['TotalKg'].iloc[-1]

    # 2. historical performances, averaged, min, max
    features['avg_total'] = previous_meet['TotalKg'].mean()
    features['max_total_ever'] = previous_meet['TotalKg'].max()
    features['min_total_ever'] = previous_meet['TotalKg'].min()

    features['avg_squat'] = previous_meet['Best3SquatKg'].mean()
    features['avg_bench'] = previous_meet['Best3BenchKg'].mean()
    features['avg_deadlift'] = previous_meet['Best3DeadliftKg'].mean()

    # 3. improvement rates over meets, 
    if len(previous_meet) > 1:
        first_total = previous_meet['TotalKg'].iloc[0]
        last_total = previous_meet['TotalKg'].iloc[-1]
        num_meets = len(previous_meet) - 1 # n meets -> n - 1 intervals

        features['total_gain_per_meet'] = (last_total - first_total) / num_meets



    

In [ ]:
def engineer_features(df):
    # already sorted by name and date from preprocessing

    lifting_data = []
    df = df.sort_values(['Name', 'Date']).reset_index(drop=True) # sort again for now, might need to modify preprocessing later

    for name, lifter_data in df.groupby('Name'): # for each lifters competition history
        if len(lifter_data) < 2: # only can predict lifters with at least two comp history
            continue
        # TODO: call prev method to create features
